In [ ]:
# 2024년 기준 종가가 밴드 하단을 터치하면 다음날 시가에 매수
# 기다렸다가 밴드의 상단을 터치하면 다음날 시가에 매도하는 전략
# 누적수익률이 가장 높은 종목 (20일, 표준편차 * 2)
df = pd.read_csv("SP500/AAPL.csv")
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index("Unnamed: 0")

def problem3(df):
    df['MA20'] = df['Close'].rolling(window = 20).mean()
    df['STD20'] = df['Close'].rolling(window = 20).std()
    df['UB'] = df['MA20'] + 2 * df['STD20']
    df['LB'] = df['MA20'] - 2 * df['STD20']
    df = df.dropna()
    
    df = df[df.index.year == 2024]
    
    cond1 = df['Close'] <= df['LB']
    cond2 = df['Close'] >= df['UB']
    
    import warnings
    warnings.filterwarnings("ignore")
    
    df['구매신호'] = cond1.shift(1).fillna(False)
    df['판매신호'] = cond2.shift(1).fillna(False)
    
    buy_price = -1
    cum_rate = 1
    
    for i in range(len(df)):
        if (df.iloc[i]['구매신호'] == True) and (buy_price == -1):
            buy_price = df.iloc[i]['Open']
    
        elif (df.iloc[i]['판매신호'] == True) and (buy_price != -1):
            sell_price = df.iloc[i]['Open']
            rate = sell_price / buy_price
            cum_rate *= rate
            buy_price = -1
    
    return cum_rate